In [1]:
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
font_name = mpl.font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name()
mpl.rc('font', family=font_name)
mpl.rcParams.update({'font.size': 22})

In [3]:
using_df = pd.read_csv("../input/서울특별시 공공자전거 대여이력 정보_2017년_1.csv", engine='python')
using_df = using_df.rename(columns={ 
    "대여소번호" : "대여대여소번호",
    "대여대여소명" : "대여대여소",
    "거치대번호" : "대여거치대",
    "반납대여소명" : "반납대여소",
    "반납 거치대번호" : "반납거치대"
})

for i in using_df.columns:
    using_df = using_df.rename(columns={i:"'" + i + "'"})

 
for i in range(1,7):
    temp = pd.read_csv(f"../input/서울특별시 공공자전거 대여이력 정보_2017년_{i+1}.csv", engine='python')
    using_df = pd.concat([using_df,temp], axis=0)


        
# for i in range(0,5):
#     temp = pd.read_csv(f"../input/서울특별시 공공자전거 대여이력 정보_2018년{i+1}.csv", engine='python')
#     using_df = pd.concat([using_df,temp], axis=0)
    
using_df = using_df.reset_index()
del using_df['index']

for i in using_df.columns:
    using_df = using_df.rename(columns={i:i.replace("'","")})
using_df.head()

In [ ]:
work_df = using_df.set_index('대여일시')

In [ ]:
work_df.head()

In [ ]:
eiyoung = using_df.groupby("대여대여소번호")["대여일시"].count().reset_index()
#이용횟수
eiyoung

In [ ]:
using_df['날짜'] = using_df['대여일시'].dt.date
nalza = using_df.groupby(["대여대여소번호", "대여대여소", "날짜"])["대여일시"].count().reset_index()
daysofusing = nalza.groupby(["대여대여소번호", "대여대여소"])["날짜"].count().reset_index()
daysofusing = daysofusing.rename(columns={"날짜": "이용일수"})

In [ ]:
usagedf = eiyoung.merge(daysofusing, on="대여대여소번호", how='left')
usagedf.head()

In [ ]:
usagedf["이용률"] = list(map(lambda x,y : x/y, usagedf["대여일시"], usagedf["이용일수"]))
usagedf.sort_values(by="이용률", ascending=True)

In [ ]:
gerchi = pd.read_excel("../input/공공자전거 대여소 운영현황(171108) (1).xlsx")
gerchi["대여소번호"] = gerchi["대여소명"].apply(lambda x:str(x).split(".")[0].strip())
gerchi = gerchi[["대여소번호", "대여소명","대여소_구"]]
gerchi = gerchi.rename(columns={"대여소_구" : "구"})
gerchi.head()

In [ ]:
gerchi2 = pd.read_excel("../input/공공자전거대여소정보_20180409.xlsx")
gerchi2.head()

gerchi_all = pd.concat([gerchi, gerchi2[["대여소번호", "대여소명", "구"]]], axis=0)
gerchi_all['대여소번호'] = gerchi_all["대여소번호"].apply(str)

In [ ]:
gerchi_all = gerchi_all.drop_duplicates(subset="대여소번호", keep="first")

In [ ]:
merged_df = usagedf.merge(gerchi_all, left_on="대여대여소번호", right_on="대여소번호", how='left')
merged_df.head()

In [ ]:
merged_df.sort_values(by="이용률", ascending=False)